In [ ]:
access_token = 'EAAH24PQU4egBADGV6Xd8EnwaCmEjvlKZCLYsymsSZBVCPBvvZBtcOnMuotunZAGh4BBvAP9dHYLAnS5J70UOIXDgEbxhgHbGXOTWGAESBfSlJGj6GDZAlDvh5KYzAztgsaOxbGSlVLaT29P00gmsSLBNGXiSZBT71mRUa5rsFo2I8ZC6d4I0oBFFZCaqbyKbbCMZD'
host = '78.46.226.67'
port = '5432'
database ='postgres'
user='kirill_k'
password='swfr35$#'

address = 'postgresql://' + user +':' + password + '@' + host +':' + port + '/' + database

In [2]:
!pip install facebook-business
import psycopg2

from tqdm import tqdm

import pandas as pd
import numpy as np

from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adcreative import AdCreative
from facebook_business.exceptions import FacebookRequestError, FacebookBadObjectError
from facebook_business.adobjects.user import User
from facebook_business.api import FacebookAdsApi
from psycopg2 import OperationalError

import json

import datetime
from datetime import timedelta, datetime, date, time
import time

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
def connection_to_database(host, port, database, user, password):
    connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
    cur = connection.cursor()
    return cur, connection

def reconnection_to_database(host, port, database, user, password, connection):
    if connection.closed > 0:
        connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
        cur = connection.cursor()
    else:
        try:
            connection.cursor().execute('SELECT 1')
        except OperationalError:
            connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
            cur = connection.cursor()
    return connection.cursor(), connection

def store_object(object_data, table):
    fields = ','.join(object_data.keys())
    values = ','.join(map(lambda x:f"%({x})s",object_data.keys()))
    query =   f"INSERT INTO {table} ({fields}) values ({values})"
    cur.execute(query, object_data)
    connection.commit()
    
def delete_object(object_id, table):
    query_to_delete = f"""delete  from {table} where id = %s"""  
    cur.execute(query_to_delete, vars=(object_id, ))
    
def store_deleted_object(object_id, object_type, account_id, table):
    today = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    query =  f"INSERT INTO {table} values (%s, %s, %s, %s)"
    cur.execute(query, vars=(object_id, account_id, object_type,today,))
    connection.commit()

def dict2json(dictionary):
    return json.dumps(dictionary, ensure_ascii=False, sort_keys=True)

def modify_object(object_data):
    for key in object_data.keys():
        if type(object_data[key]) in (list, dict):
            object_data[key] = dict2json(object_data[key])
    object_data['recording_date'] = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    
    return object_data

In [5]:
print('start processing \t' + datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))

FacebookAdsApi.init(access_token=access_token)

try:
    cur.close()
    cur, connection = connection_to_database(host, port, database, user, password)
except NameError:
    cur, connection = connection_to_database(host, port, database, user, password)

me = User(fbid='me')
accounts = list(me.get_ad_accounts())
accounts = list(map(lambda x:x.export_all_data(), accounts))
account_ids = list(map(lambda x:x['account_id'], accounts))


object_type = 'creative'
table = 'property_' + object_type + 's'

cur.execute(f""" CREATE TABLE if not exists {table}(
    id VARCHAR(50) unique primary key,    account_id VARCHAR(50),     name VARCHAR(200), title TEXT, 
    body TEXT, object_type VARCHAR(20), call_to_action_type VARCHAR(20), status VARCHAR(20), 
    effective_instagram_story_id VARCHAR(50), effective_object_story_id VARCHAR(50), object_story_id VARCHAR(50), 
    url_tags TEXT, effective_authorization_category VARCHAR(20), authorization_category VARCHAR(20),
    enable_launch_instant_app BOOLEAN, use_page_actor_override BOOLEAN, enable_direct_install BOOLEAN,
    image_hash VARCHAR(200), image_url TEXT, image_crops JSONB, video_id  VARCHAR(50), instagram_actor_id VARCHAR(50), 
    instagram_permalink_url TEXT, object_store_url TEXT, platform_customizations JSONB,
    object_story_spec JSONB, thumbnail_url TEXT, product_set_id VARCHAR(50), applink_treatment VARCHAR(50),
    template_url_spec JSONB, portrait_customizations JSONB, link_og_id VARCHAR(50), object_url TEXT,
    playable_asset_id VARCHAR(50), adlabels JSONB,
    recording_date timestamp without time zone)""")
connection.commit()

cur.execute(f""" CREATE TABLE if not exists deleted_objects(
    object_id VARCHAR(50) unique primary key,    account_id VARCHAR(50),     
    object_type VARCHAR(20),    recording_date timestamp without time zone)""")
connection.commit()

fields = list(AdCreative.Field.__dict__.keys())
exclude_list = ['__module__', '__doc__',  'call_to_action', 'image_file', 'is_dco_internal']
fields = list(set(fields)-set(exclude_list))

query = """SELECT DISTINCT ad.account_id, ad.id as ad_id, ad.creative ->> 'id' as creative_id, 
True as has_not_data
FROM property_ads ad
LEFT JOIN property_creatives creative on creative.id = ad.creative ->> 'id'
LEFT JOIN deleted_objects on ad.creative ->> 'id' = deleted_objects.object_id
WHERE creative.id IS NULL and deleted_objects.object_id IS NULL"""

creative_list = pd.read_sql(query, connection)
data = pd.read_sql(query, connection)
data = data[data['account_id'].isin(account_ids)]
creatives = dict(zip(list(data['creative_id']), data['has_not_data']))
accounts =  dict(zip(list(data['creative_id']), list(data['account_id'])))

print('start uploading \t' + datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))

for creative_id in tqdm(creatives.keys()):
    try:
        creative_data = AdCreative(creative_id).api_get(fields = fields).export_all_data()
        creative_data = modify_object(creative_data)
        cur, connection = reconnection_to_database(host, port, database, user, password, connection)
        if creatives[creative_id]:
            store_object(creative_data, table)
        else:
            delete_object(creative_id, table)
            store_object(creative_data, table)
        connection.commit()
        
    except FacebookRequestError as error:
        code = error._body['error']['code']
        subcode = error._body['error'].get('error_subcode', None)
        if code == 100 and (subcode == 33 or subcode == 1487221):
            cur, connection = reconnection_to_database(host, port, database, user, password, connection)
            store_deleted_object(creative_id, object_type, accounts[creative_id], 'deleted_objects')

        #print('An unknown error occurred for adset ' + adset_id)

print('start refreshing \t' + datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))        

cur, connection = reconnection_to_database(host, port, database, user, password, connection)
query = """REFRESH MATERIALIZED VIEW mview_audit_by_offers"""
cur.execute(query)
connection.commit()

print('end processing \t' + datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))

start processing 	2020-07-06 15:33:02



  0%|          | 0/13391 [00:00<?, ?it/s]

start uploading 	2020-07-06 15:33:09


Streaming output truncated to the last 5000 lines.
100%|██████████| 13391/13391 [1:29:45<00:00,  2.49it/s]


start refreshing 	2020-07-06 17:02:55


ProgrammingError: ignored